In [2]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items') # Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='shivam@123',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/ASUS/OneDrive/Desktop/Ecommerce'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processing sellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processing products.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
prod

In [2]:
import pandas as pd
import mysql.connector

db = mysql.connector.connect(
    host = "localhost",
    username= "root",
    password = "shivam@123",
    database = "ecommerce")

cur = db.cursor()

# List all unique cities where customers are located.

In [20]:
query = """select distinct customer_city from customers """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ["City"])
df

,City
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas
...,...
4114,siriji
4115,natividade da serra
4116,monte bonito
4117,sao rafael


# Count the number of orders placed in 2017.

In [19]:
query = """ select count(order_id) from orders where year(order_purchase_timestamp) = 2017 """

cur.execute(query)
data = cur.fetchall()

"total orders placed in 2017 are", data[0][0]

('total orders placed in 2017 are', 45101)

# Find the total sales per category.

In [39]:
query = """ select upper(products.product_category) category, round(sum(payments.payment_value),2) sales 
from products join order_items on products.product_id = order_items.product_id
join payments on payments.order_id = order_items.order_id group by category """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Category", "sales"])
df.head(10)

,Category,sales
0,PERFUMERY,506738.66
1,FURNITURE DECORATION,1430176.39
2,TELEPHONY,486882.05
3,BED TABLE BATH,1712553.67
4,AUTOMOTIVE,852294.33
5,COMPUTER ACCESSORIES,1585330.45
6,HOUSEWARES,1094758.13
7,BABIES,539845.66
8,TOYS,619037.69
9,FURNITURE OFFICE,646826.49


# Calculate the percentage of orders that were paid in installments.

In [28]:
query = """ select count(case when payment_installments >= 1 then 1 end) * 100/count(*)
as payment_in_installment from payments; """

cur.execute(query)
data = cur.fetchall()

"Percentage of orders that were paid in installment is", data[0][0]

('Percentage of orders that were paid in installment is', Decimal('99.9981'))

# Count the number of customers from each state.

In [38]:
query = """ select customer_state, count(customer_state) from customers group by customer_state """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["State", "Customer_count"])
df.head(10)

,State,Customer_count
0,SP,41746
1,SC,3637
2,MG,11635
3,PR,5045
4,RJ,12852
5,RS,5466
6,PA,975
7,GO,2020
8,ES,2033
9,BA,3380


# Calculate the number of orders per month in 2018.

In [36]:
query = """ select month(order_purchase_timestamp) as months, count(*) from orders
where year(order_purchase_timestamp) = 2018 group by month(order_purchase_timestamp) order by months; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Month", "total_orders"])
df

,Month,total_orders
0,1,7269
1,2,6728
2,3,7211
3,4,6939
4,5,6873
5,6,6167
6,7,6292
7,8,6512
8,9,16
9,10,4


# Retrieve the first 10 orders placed by customers in 2017.

In [37]:
query = """ select order_id, order_purchase_timestamp from orders where year(order_purchase_timestamp) = 2017
order by order_purchase_timestamp limit 10; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Order_id", "Date&Time"])
df

,Order_id,Date&Time
0,ec7a019261fce44180373d45b442d78f,2017-01-05 11:56:06
1,b95a0a8bd30aece4e94e81f0591249d8,2017-01-05 12:01:20
2,38bcb524e1c38c2c1b60600a80fc8999,2017-01-05 12:06:36
3,7a18a504c1a4b32d883e68de2e1a7db0,2017-01-05 12:09:08
4,6acecf438369055d9243e121045cca74,2017-01-05 12:11:23
5,34bf4feda1e203af64692d97c6950c39,2017-01-05 12:14:58
6,40599d3d28b75746952ded75566637b9,2017-01-05 13:01:48
7,ce86fa5a5108884726a2244bcae51ae6,2017-01-05 13:29:03
8,7004296aa0256632eaddc171edaf727f,2017-01-05 13:31:22
9,cce1b8a1c5f8b1d224e19628299c8f54,2017-01-05 13:33:45


# Find the top 5 cities with the most customers.

In [40]:
query = """ select customer_city, count(*) as customer_count from customers 
group by customer_city order by customer_count desc limit 5; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["City", "Customer_count"])
df

,City,Customer_count
0,sao paulo,15540
1,rio de janeiro,6882
2,belo horizonte,2773
3,brasilia,2131
4,curitiba,1521


# Count the number of orders for each order status.

In [42]:
query = """ select order_status, count(*) as order_count from orders
group by order_status; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Order_status", "Order_count"])
df

,Order_status,Order_count
0,delivered,96478
1,invoiced,314
2,shipped,1107
3,processing,301
4,unavailable,609
5,canceled,625
6,created,5
7,approved,2


# Find the highest order value (sum of price + freight_value) for a single order.


In [45]:
query = """ select max(price + freight_value) as order_value from order_items; """

cur.execute(query)
data = cur.fetchall()

"highest Order Value is", data[0][0]

('highest Order Value is', 6929.309997558594)

# Find the total revenue generated from each payment type.


In [49]:
query = """ select payment_type, sum(payment_value) as revenue from payments group by payment_type; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Payment_type", "Revenue"])
pd.set_option('display.float_format', '{:,.6f}'.format)
df

,Payment_type,Revenue
0,credit_card,"12,542,084.190177"
1,UPI,"2,869,361.269494"
2,voucher,"379,436.870396"
3,debit_card,"217,989.789988"
4,not_defined,0.000000


# Find the total profit generated from each payment type.

In [50]:
query = """ select p.payment_type, sum(p.payment_value)-sum(o.price + o.freight_value) as profit
from payments as p inner join order_items as o on p.order_id = o.order_id 
group by p.payment_type; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Payment_type", "Profit"])
pd.set_option('display.float_format', '{:,.6f}'.format)
df

,Payment_type,Profit
0,credit_card,"2,865,113.339049"
1,voucher,"-379,208.910519"
2,UPI,"1,217,459.304107"
3,debit_card,"38,227.119943"


# Find the total number of products sold in each city.

In [53]:
query = """ select temp2.customer_city, count(p.product_id) as count from products as p
inner join (select temp.*, product_id from order_items as ot 
inner join (select c.*, o.order_id from customers as c 
inner join orders as o on c.customer_id = o.customer_id) as temp
on ot.order_id = temp.order_id) as temp2 on p.product_id = temp2.product_id
group by temp2.customer_city order by count desc; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Customer_city", "No_of_Product"])
df.head(10)

,Customer_city,No_of_Product
0,sao paulo,17808
1,rio de janeiro,7837
2,belo horizonte,3144
3,brasilia,2392
4,curitiba,1751
5,campinas,1654
6,porto alegre,1612
7,salvador,1412
8,guarulhos,1329
9,sao bernardo do campo,1060


# Find the customers who have not placed any orders.

In [55]:
query = """ select c.customer_id, o.order_id from customers as c 
left join orders as o on c.customer_id = o.customer_id
where o.order_id is NULL; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Customer_id", "Order_id"])
df

,Customer_id,Order_id
0,06b8999e3fba9a1fbc88172z30ba8ba7,None
1,06b8999e4aba9a1cbc88172z50ba8ba7,None
2,06b8999e3fba9f1fbc88172z00ba8ba7,None
3,06b8999e2fba9a1fbc88172z00ba8ba7,None
4,06b8999e2fba9a4fbc88172z60ba8ba7,None


# Find the product category with the most products and display the product_category along with the count.

In [2]:
query = """ select product_category, count(*) as count from products
group by product_category order by count desc; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Product_category", "Product_count"])
df.head(10)

,Product_category,Product_count
0,bed table bath,6058
1,sport leisure,5734
2,Furniture Decoration,5314
3,HEALTH BEAUTY,4888
4,housewares,4670
5,automotive,3800
6,computer accessories,3278
7,toys,2822
8,Watches present,2658
9,telephony,2268


# Find the 2nd highest payment value using subquery.

In [3]:
query = """ select  max(payment_value) from payments as p1 where payment_value
not in(select max(payment_value) from payments); """

cur.execute(query)
data = cur.fetchall()

"Second highest payment_value is:", data[0][0]

('Second highest payment_value is:', 7274.88)

# List customers who placed orders after a 1 feb 2018.

In [2]:
query = """ select c.customer_id, o.order_id, o.order_purchase_timestamp from
customers as c inner join orders as o on c.customer_id = o.customer_id
where date(o.order_purchase_timestamp) > '2018-02-01'; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Cuatomer_id", "Order_id", "Order_purchase_timestamp"])
df.head(10)

,Cuatomer_id,Order_id,Order_purchase_timestamp
0,df0aa5b8586495e0ddf6b601122e43a1,72bd19ef4fa285334b95bda01a3718c7,2018-05-09 13:01:24
1,3f6ede29d4c69cd3316d2035b6cec1fb,23c0fb66f3afa483b302cc740b14f17b,2018-05-16 23:45:47
2,3ccb27f9c7ee205656acef98c19fc061,a8c3124b7f912401d702018ae0c02b05,2018-04-16 20:46:53
3,154e666b681104319714a99a3ab71e44,8ae61eab795bd8831fe171750551feb1,2018-07-31 00:17:40
4,ae8db0691449a44352e7d535ddf78c5e,f11b36b3bc7bacf06deef862ed611f02,2018-05-15 09:10:20
5,4d5110d0a2b5239cb983610da1d18774,784b0448b2075a95afcf2a55310b7238,2018-05-16 20:57:48
6,852947b57caab58c544343592f5e06d2,cdf2839409d0f62e2b2e61b62b72dc16,2018-02-14 10:26:22
7,1004c05877ddf8196ad1341ca602e628,33cf8e9da9865079f64965b83061baae,2018-03-20 09:25:27
8,4b003ee1eabaffe8ff8e6d75394a9b42,760312f5735ee281f01e56d5ddee05e3,2018-06-11 15:32:06
9,71e31bd9c37b8a1cacc557b87b3e50cb,56dc7f1b8195b7fbdd6479f055a203bb,2018-03-16 15:49:44


# Retrieve orders with payment values greater than the average payment value.

In [7]:
query = """ select order_id, payment_value from payments
where payment_value > (select avg(payment_value) from payments); """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Order_id", "Payment_value",])
df.head(10)

,Order_id,Payment_value
0,1f78449c87a54faf9e96e88ba1491fa9,341.09
1,d88e0d5fa41661ce03cf6cf336527646,188.73
2,12e5cfe0e4716b59afb0e0f4a3bd6570,157.45
3,8ac09207f415d55acff302df7d6a895c,244.15
4,4214cda550ece8ee66441f459dc33a8c,170.57
5,4d680edbaa7d3d9bed69532957368a03,353.09
6,8cd68144cdb62dc0d60848cf8616d2a4,330.66
7,d0a945f85ba1074b60aac97ade7e240e,541.00
8,5d9c5817e278892b7498d90bfa28ade8,290.16
9,b69b9260e79a0da00e15f48de1bd2524,283.34


# List orders from customers in the city "Sao Paulo" along with the latitude and longitude.

In [2]:
query = """ select o.order_id, temp.* from orders o inner join 
(select c.customer_id, g.geolocation_lat, g.geolocation_lng from customers c
inner join geolocation g on c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
where g.geolocation_city = 'sao paulo') as temp
on o.customer_id = temp.customer_id; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Order_id", "customer_id", "geolocation_lat", "geolocation_lng"])
df.head(10)

,Order_id,customer_id,geolocation_lat,geolocation_lng
0,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5369,-46.6335
1,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5373,-46.6339
2,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5425,-46.6353
3,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5425,-46.6353
4,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5425,-46.6353
5,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5407,-46.6340
6,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5425,-46.6353
7,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5430,-46.6349
8,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5442,-46.6363
9,1eb7606f2ad568b8303fb24c8ab1e2cd,bac56672aefaf3167d4d46c1ed0fdc92,-23.5425,-46.6353


# Retrieve products that have been sold more than 10 times.

In [3]:
query = """ select product_id from order_items 
group by product_id having COUNT(*) > 10; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Product_id",])
df.head(10)

,Product_id
0,ac6c3623068f30de03045865e4e10089
1,d63c1011f49d98b976c352955b1c4bea
2,99a4788cb24856965c36a24e339b6058
3,368c6c730842d78016ad823897a372db
4,4fa33915031a8cde03dd0d3e8fb27f01
5,5ed9eaf534f6936b51d0b6c5e4d5c2e9
6,553e0e7590d3116a072507a3635d2877
7,57d79905de06d8897872c551bfd09358
8,5d7c23067ed3fc8c6e699b9373d5890b
9,5a419dbf24a8c9718fe522b81c69f61a


# List order IDs and product names ordered by total price.

In [8]:
query = """ select distinct o.order_id, p.product_category, oi.price + oi.freight_value as total_price
from orders o inner join order_items oi on o.order_id = oi.order_id
inner join products p on oi.product_id = p.product_id
order by total_price desc; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Order_id","Product_category","total_price"])
df.head(10)

,Order_id,Product_category,total_price
0,0812eb902a67711a1cb742b3cdaa65ae,housewares,6929.309998
1,fefacc66af859508bf1a7934eab1e97f,PCs,6922.210007
2,f5136e38d1a14a4dbd87dff67da82701,Art,6726.660004
3,a96610ab360d42a2e5335a3998b4718a,electrostile,4950.339996
4,199af31afc78c699f0dbf71fb178d4d4,electrostile,4764.339996
5,8dbc85d1447242f3b127dda390d56e19,PCs,4681.779999
6,426a9742b533fc6fed17d1fd6d143d7e,musical instruments,4513.320114
7,80dfedb6d17bf23539beeef3c768f4d7,electrostile,4194.759995
8,68101694e5c5dc7330c91e1bbc36214f,Games consoles,4175.260231
9,b239ca7cd485940b31882363b52e6674,sport leisure,4163.510002


# Find customers who have placed more than 1 orders.

In [6]:
query = """ select customer_id, count(order_id) as count from orders
group by customer_id having count > 1 """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Customer_id","Order_count"])
df.head(10)

,Customer_id,Order_count
0,9ef432eb6251297304e76186b10a928d,2
1,b0830fb4747a6c6d20dea0b8c802d7ef,2
2,41ce2a54c0b03bf3443c3d931a367089,2
3,f88197465ea7920adcdbec7375364d82,2
4,8ab97904e6daea8866dbdbc4fb7aad2c,2
5,503740e9ca751ccdda7ba28e9ab8f608,2
6,ed0271e0b7da060a393796590e7b737a,2
7,9bdf08b4b3b52b5526ff42d37d47f222,2
8,f54a9f0e6b351c431402b8461ea51999,2
9,31ad1d1b63eb9962463f764d4e6e0c9d,2


# List all order IDs, product IDs, time, and price for orders placed after '2018-01-01' with a product price over 200.

In [7]:
query = """ select o.order_id, oi.product_id, o.order_purchase_timestamp, oi.price
from orders o 
inner join order_items oi on o.order_id = oi.order_id
where o.order_purchase_timestamp > '2018-01-01' and oi.price > 200; """

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["Order_id","Product_count", "Timestamp", "Price"])
df.head(10)

,Order_id,Product_count,Timestamp,Price
0,1d251ab94983c4adb11e4b168abb1439,fb92dc4602bd9224e0e267496748afc5,2018-03-07 23:46:19,219.00
1,04040ee654b248cdc512a68ecc83e4cc,cc7d8d232e6f5f27b70be81b68e27d6e,2018-03-27 22:14:59,259.90
2,2b72a32a2c3e93fa260ca7655073886f,cc3181a81384b6fc34602d49cb6068a6,2018-04-18 15:40:36,1148.00
3,0438fc3e115633e11a93c878a1591016,afeeea6271148ee1bb15173b8187c431,2018-08-11 20:58:44,249.00
4,197edbaa61a8a349f382553fd2180ee1,22a8168cf1f1c6347da8d29863b7f75e,2018-07-02 13:58:59,296.50
5,142ac5357e541e5881d26fc5eb5f3754,99e28f7c171b6945dc991d1178ffedfe,2018-04-30 20:24:33,279.99
6,011b142c9e082a5c1d10e0a88cd9c8e8,4cb33e9787e8a38b82bd68e70f5f4700,2018-03-23 11:43:13,243.37
7,05ae7d6bd78e2a61f4c0076a9d690aa8,c80124276571cd4bef53cf35d9b36c5c,2018-07-20 12:25:03,219.00
8,2cb414479754a8e1ed07eed0ccfc0778,98600823ba81f3ba76386dac7c31ee06,2018-06-03 09:59:42,399.00
9,1f69e4870ffb8313df2fef3df0af2718,b5f0fcccccb9b8be8914e92fd322804f,2018-03-06 17:12:14,380.00


# Identify the top 3 customers who spent the most money in each year.